In [1]:
import pandas as pd, numpy as np

In [2]:
train = pd.read_csv('data/train.csv', index_col='time')
test = pd.read_csv('data/test.csv', index_col='time')

In [3]:
X_train, y_train, X_valid, y_valid = train[[col for col in train.columns if col!='severity']], train['severity'],\
                                     test[[col for col in train.columns if col!='severity']], test['severity']

In [4]:
from keras_text.models import SentenceModelFactory
from keras_text.models import YoonKimCNN, AttentionRNN, StackedRNN
from keras_text.processing import WordTokenizer
from keras_text.data import Dataset

/home/denis/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
tokenizer = WordTokenizer()
tokenizer.build_vocab(X_train['message_encoding'].values, verbose=1)

10364/10364 [==============================] - 45s 4ms/step


In [9]:
# Pad max sentences per doc to 500 and max words per sentence to 200.
# Can also use `max_sents=None` to allow variable sized max_sents per mini-batch.
factory = SentenceModelFactory(10, tokenizer.token_index, max_sents=500, max_tokens=200, embedding_type='glove.6B.100d')
word_encoder_model = AttentionRNN()
sentence_encoder_model = AttentionRNN()

In [7]:
# Allows you to compose arbitrary word encoders followed by sentence encoder.
model = factory.build_model(word_encoder_model, sentence_encoder_model)
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

INFO:keras_text.embeddings:Building embeddings index...
INFO:keras_text.embeddings:Done
INFO:keras_text.embeddings:Loading embeddings for all words in the corpus


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 500, 200)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 500, 100)          7340700   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 500, 100)          60400     
_________________________________________________________________
attention_layer_2 (Attention (None, 100)               10200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 7,412,310
Trainable params: 7,412,310
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(X_train['message_encoding'].values, y_train)

ValueError: Error when checking input: expected input_2 to have 3 dimensions, but got array with shape (10364, 1)

In [ ]:
tokenizer.token_generator()